# GCN & Random-GCN(RGCN) 
For:

Learning from the Dark: Boosting Graph Convolutional Neural Networks wit Diverse Negative Samples.

This code can be run directly on colab

Random sampling to get negative samples refer to this article:

[HOW TO FIND YOUR FRIENDLY NEIGHBORHOOD:
GRAPH ATTENTION DESIGN WITH SELF-SUPERVISION](https://openreview.net/forum?id=Wi5KUNlqWty)

# 0 Preparing

In [ ]:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install -q torch-geometric

!pip install dppy

import torch
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from dppy.finite_dpps import FiniteDPP
from dppy.utils import example_eval_L_linear

     |████████████████████████████████| 3.0 MB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 5.1 MB/s 
     |████████████████████████████████| 222 kB 5.3 MB/s 
     |████████████████████████████████| 376 kB 47.9 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
     |████████████████████████████████| 83 kB 1.7 MB/s 


## 0.1 Download data

In [ ]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.datasets import KarateClub

# dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
dataset = Planetoid(root='data/Planetoid', name='Citeseer', transform=NormalizeFeatures())
# dataset = Planetoid(root='data/Planetoid', name='Pubmed')

print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Processing...
Done!

Dataset: Citeseer():
Number of graphs: 1
Number of features: 3703
Number of classes: 6

Data(edge_index=[2, 9104], test_mask=[3327], train_mask=[3327], val_mask=[3327], x=[3327, 3703], y=[3327])
Number of nodes: 3327
Number of edges: 9104
Average node degree: 2.74
Number of training nodes: 120
Training node label rate: 0.04
Contains isolated nodes: True
Contains self-loops: False
Is undirected: True


# 1 Data pre-processing
## 1.1 Calculating the maximum connected subgraph

In [ ]:
from torch_geometric.utils import to_networkx, from_networkx
import networkx as nx

# transform data from torch_geometric to networkx
G = to_networkx(data, to_undirected=True)
# Get the index of all points of the max-connected subgraph
Subnode = max(nx.connected_components(G), key=len)
print(len(Subnode))
# Get the subgraph
SubGraph = G.subgraph(Subnode)
SubnodeList = list(Subnode)
print(SubnodeList)

2120
[1, 5, 8, 10, 12, 13, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 38, 39, 40, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 61, 62, 65, 66, 69, 70, 72, 75, 76, 77, 78, 79, 80, 81, 83, 84, 87, 88, 90, 91, 92, 93, 95, 96, 98, 99, 100, 101, 103, 104, 105, 106, 107, 110, 113, 114, 115, 118, 119, 122, 123, 124, 126, 128, 130, 131, 132, 134, 135, 136, 137, 138, 142, 144, 147, 148, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 162, 167, 168, 169, 170, 172, 173, 177, 178, 180, 181, 184, 186, 188, 189, 190, 191, 194, 195, 197, 198, 200, 201, 203, 204, 205, 206, 208, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 224, 226, 227, 228, 229, 230, 231, 232, 234, 236, 237, 240, 241, 242, 243, 244, 246, 247, 249, 250, 252, 253, 254, 255, 258, 259, 260, 263, 265, 266, 267, 268, 269, 272, 273, 274, 280, 285, 286, 287, 289, 292, 293, 294, 298, 300, 302, 303, 304, 307, 308, 311, 312, 313, 314, 316, 317, 318, 319, 321, 322, 325, 330, 331, 332, 333,

## 1.2 Transform data from networkx to torch_geometric

In [ ]:
PyTSubGraph = from_networkx(SubGraph)
SubGraph = to_networkx(PyTSubGraph, to_undirected=True)

PyTSubGraph.test_mask = data.test_mask[SubnodeList]
PyTSubGraph.train_mask = data.train_mask[SubnodeList]
PyTSubGraph.val_mask = data.val_mask[SubnodeList]
PyTSubGraph.x = data.x[SubnodeList]
PyTSubGraph.y = data.y[SubnodeList]
print(PyTSubGraph)

Data(edge_index=[2, 7358], test_mask=[2120], train_mask=[2120], val_mask=[2120], x=[2120, 3703], y=[2120])


In [ ]:
# Get number of nodes
num_nodes = PyTSubGraph.num_nodes
# Get edge_index
edge_index = PyTSubGraph.edge_index
edge_value = torch.ones(edge_index.size(1), device=edge_index.device)
# Transforming the adjacency matrix from a sparse transpose matrix to a dense matrix
adjdense = torch.sparse.FloatTensor(edge_index, edge_value, torch.Size([num_nodes,num_nodes])).to_dense()
print(adjdense.shape)

torch.Size([2120, 2120])


# 3 Model Definition
# 3.1 Basic GCN MessagePassing Layer

In [ ]:
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_scatter import gather_csr, scatter
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3-5: Start propagating messages.
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j, edge_index, size):
        # x_j has shape [E, out_channels]
        # edge_index has shape [2, E]

        # Step 3: Normalize node features.
        row, col = edge_index
        deg = degree(row, size[0], dtype=x_j.dtype)  # [N, ]
        deg_inv_sqrt = deg.pow(-0.5)   # [N, ]
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        return norm.view(-1, 1) * x_j
            
    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 5: Return new node embeddings.
        return aggr_out

## 3.2 Defining the MAD

In [ ]:
def MAD(x):
  x_norm = x / x.norm(dim=1)[:, None]
  dist = 1-torch.mm(x_norm, x_norm.transpose(0,1))
  one1 = torch.ones_like(dist)
  zreo1 = torch.zeros_like(dist)
  dist_1 = torch.where(dist > 0, one1, zreo1)
  D = dist.sum(0)/dist_1.sum(0)
  one = torch.ones_like(D)
  zero = torch.zeros_like(D)
  D_1 = torch.where(D > 0, one, zero)
  Mad = D.sum()/D_1.sum()
  return Mad.detach().numpy()


## 3.3 Defining the GCN model

In [ ]:
import torch
import torch.nn.functional as F
class GCN(torch.nn.Module):
    def __init__(self, num_layers, hidden_channels):
        super(GCN, self).__init__()
        
        self.num_layers = num_layers
        self.GCNs = torch.nn.ModuleList()
        for layer in range(self.num_layers-1):
            if layer == 0:
                self.GCNs.append(GCNConv(dataset.num_features, hidden_channels))
            else:
                self.GCNs.append(GCNConv(hidden_channels, hidden_channels))
        self.GCNs.append(GCNConv(hidden_channels, dataset.num_classes))

    def forward(self, x, edge_index):
        for layer in range(self.num_layers):
          x = self.GCNs[layer](x, edge_index)
          if layer < (self.num_layers-1):
            x = x.relu()
            x = F.dropout(x, p=0.5, training=self.training)
        return x

## 3.4 Defining the RGCN model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import time


class RanGCN(torch.nn.Module):
    def __init__(self, hidden_channels,num_layers,adjdense,num_nodes):
        super(RanGCN, self).__init__()
        # torch.manual_seed(12345)

        self.num_layers = num_layers
        self.CONVs = torch.nn.ModuleList()
        # RandomGCN list
        self.RCONVs = torch.nn.ModuleList()
        # Negative Rates list
        self.NegRates = nn.ParameterList()
        for layer in range(self.num_layers-1):
            if layer == 0:
                self.CONVs.append(GCNConv(dataset.num_features, hidden_channels))
                self.RCONVs.append(GCNConv(dataset.num_features, hidden_channels))
                self.NegRates.append(nn.Parameter(torch.FloatTensor([1])))
            else:
                self.CONVs.append(GCNConv(hidden_channels, hidden_channels))
                self.RCONVs.append(GCNConv(hidden_channels, hidden_channels))
                self.NegRates.append(nn.Parameter(torch.FloatTensor([1])))
        self.CONVs.append(GCNConv(hidden_channels, dataset.num_classes))
        self.RCONVs.append(GCNConv(hidden_channels, dataset.num_classes))
        self.NegRates.append(nn.Parameter(torch.FloatTensor([1])))

        self.NegRate = 1
        self.adjdense = adjdense
        self.num_nodes = num_nodes

    def forward(self, x, edge_index,Ranedge_index_list,train_neg_rate):
        for layer in range(self.num_layers):
          posi_x = self.CONVs[layer](x, edge_index)
          if len(Ranedge_index_list) < self.num_layers:
            #During Train: Generate Ranedge_index_list
            RanMatrix = self.GetRanMatrix()
            #change dense back to sparse
            Ranedge_index = torch.nonzero(RanMatrix).t()
            Ranedge_index_list.append(Ranedge_index)
          else:
            #During Test: Pass Ranedge_index_list directly to save time
            Ranedge_index = Ranedge_index_list[layer]

          nega_x = self.RCONVs[layer](x, Ranedge_index)
          
          if train_neg_rate: #NegRate is a trainable parameter
            x = posi_x - self.NegRates[layer] * nega_x
          else:
            x = posi_x - self.NegRate * nega_x

          if layer < (self.num_layers-1):
            x = x.relu()
            x = F.dropout(x, p=0.5, training=self.training)
        return x,Ranedge_index_list
       
    # Randomly generate Negative samples
    def GetRanMatrix(self):
        RanMatrix = torch.zeros((self.num_nodes,self.num_nodes))
        Indexlist = range(self.num_nodes)
        for i in range(self.num_nodes):
          # The number of samples is the same as the number of neighbors of the current node
          sampleAmount = int(self.adjdense[i].sum().item()) + 1
          # Randomly generate neighbors of the current node as Negative samples
          RanIndex = random.sample(Indexlist, sampleAmount)
          RanMatrix[i][RanIndex] = 1
        return RanMatrix
        

#4 Training
## 4.1 GCN

In [ ]:
import time

def test():
      model.eval()
      out = model(PyTSubGraph.x, PyTSubGraph.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[PyTSubGraph.test_mask] == PyTSubGraph.y[PyTSubGraph.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(PyTSubGraph.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc

#----------------
# Set training parameters
num_layers = 5
Runs = 11
Epochs = 201
#-----------------

for num in range(1,Runs):
  
  model = GCN(num_layers=num_layers,hidden_channels=16)
  if num == 1 :
    print(model)
  print("num_layers",num_layers)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
  criterion = torch.nn.CrossEntropyLoss()
  for epoch in range(1,Epochs):
    start = time.time()
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(PyTSubGraph.x, edge_index)  # Perform a single forward pass.
    loss = criterion(out[PyTSubGraph.train_mask], PyTSubGraph.y[PyTSubGraph.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    if epoch >150 and  epoch % 10 == 0:
      test_acc = test()
      print(f'num: {num:03d}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Acc: {test_acc:.4f}, MAD: {MAD(out):.4f}')
  print("-------------------")


GCN(
  (GCNs): ModuleList(
    (0): GCNConv(
      (lin): Linear(in_features=3703, out_features=16, bias=True)
    )
    (1): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (2): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (3): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (4): GCNConv(
      (lin): Linear(in_features=16, out_features=6, bias=True)
    )
  )
)
num_layers 5
num: 001, Epoch: 160, Loss: 1.7817, Acc: 0.2308, MAD: 0.0999
num: 001, Epoch: 170, Loss: 1.6689, Acc: 0.3756, MAD: 0.6074
num: 001, Epoch: 180, Loss: 1.3923, Acc: 0.3680, MAD: 0.9656
num: 001, Epoch: 190, Loss: 1.2727, Acc: 0.2821, MAD: 0.8518
num: 001, Epoch: 200, Loss: 1.1964, Acc: 0.4027, MAD: 0.8389
-------------------
num_layers 5
num: 002, Epoch: 160, Loss: 0.7632, Acc: 0.4540, MAD: 0.7201
num: 002, Epoch: 170, Loss: 0.7181, Acc: 0.4842, MAD: 0.7059
num: 002, Epoch: 180, Loss: 0.6990, Acc: 0.4465, MAD:

##4.2 RGCN

In [ ]:
import sys

def test(Ranedge_index_list,train_neg_rate):
      model.eval()
      out,_ = model(PyTSubGraph.x, edge_index,Ranedge_index_list,train_neg_rate)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[PyTSubGraph.test_mask] == PyTSubGraph.y[PyTSubGraph.test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / int(PyTSubGraph.test_mask.sum())  # Derive ratio of correct predictions.
      return test_acc

#----------------
# Set training parameters
num_layers = 5
Runs = 11
Epochs = 201
#Set whether NegRate is a trainable parameter
#if False:
#   NegRate = 1
train_neg_rate = False
#-----------------

for num in range(1,Runs):
  model = RanGCN(hidden_channels=16,
          num_layers=num_layers,
          adjdense =adjdense,
          num_nodes=num_nodes)
  if num == 1 :
    print(model)
  print("num_layers",num_layers)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
  criterion = torch.nn.CrossEntropyLoss()
  for epoch in range(1, Epochs):
      start = time.time()
      Ranedge_index_list = []
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out,Ranedge_index_list = model(PyTSubGraph.x, edge_index,Ranedge_index_list,train_neg_rate)  
      loss = criterion(out[PyTSubGraph.train_mask], PyTSubGraph.y[PyTSubGraph.train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()
      end = time.time()

      if epoch == 1:
          print("---------------")
          print("Ranedge_index_shape",Ranedge_index_list[0].shape)
          print("Epoch time:",end-start)
          print("---------------")
      if epoch % 20 == 0:
          test_acc = test(Ranedge_index_list,train_neg_rate)
          print(f'num: {num:03d}, Epoch: {epoch:03d}, Loss: {loss:.4f},Test Accuracy: {test_acc:.4f},MAD: {MAD(out):.4f}')


RanGCN(
  (CONVs): ModuleList(
    (0): GCNConv(
      (lin): Linear(in_features=3703, out_features=16, bias=True)
    )
    (1): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (2): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (3): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (4): GCNConv(
      (lin): Linear(in_features=16, out_features=6, bias=True)
    )
  )
  (RCONVs): ModuleList(
    (0): GCNConv(
      (lin): Linear(in_features=3703, out_features=16, bias=True)
    )
    (1): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (2): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (3): GCNConv(
      (lin): Linear(in_features=16, out_features=16, bias=True)
    )
    (4): GCNConv(
      (lin): Linear(in_features=16, out_features=6, bias=True)
    )
  )
  (NegRates): ParameterList(
      (0): Parameter conta